In [100]:
import numpy as np
import scipy.stats.qmc as qmc

# Exericse sheet 7

## Exercise 2 (Two simple QMC test problems)

In [101]:
generating_vectors = {
    # Source: https://wsc.project.cwi.nl/woudschoten-conferences/2016-woudschoten-conference/FKtalk1.pdf, Slide 14
    2: np.array([1, 19])
    # Source: https://people.cs.kuleuven.be/~dirk.nuyens/qmc-generators/LATSEQ/exew_base2_m20_a3_HKKN.txt
    10: np.array([1, 364981, 245389, 97823, 488939, 62609, 400749, 385317, 21281, 223487])
}

In [ ]:
def f(s: int, points: np.array) -> np.array:
    x = points.copy()
    C = (2 + 1 / (2 * s)) ** s

    x = x ** (1 + 1 / (2 * s))

    return C * np.prod(x, axis=1)

def g(s: int, points: np.array) -> np.array:
    x = points.copy()
    
    x = (x - 1 / 2) / np.arange(1, s + 1) ** 2
    
    return 1 + np.sum(x, axis=1)

In [151]:
def generate_lattice_points(
    z,  # generating vector
    m,  # 2 ** m number of points
    previous_lattice=np.array([]),  # reuse points from previous lattice
):
    def generate(index):
        return np.modf(
            np.reshape(index, (-1, 1)) * z / N
        )[0]

    # Set number of points.
    N = 2 ** m
    
    rows = np.arange(0, N, 1)

    if previous_lattice.size == 0:
        # Generate first lattice.
        new_lattice = generate(rows)
    else:
        new_lattice = np.empty([N, z.size])
        
        even_rows = rows[rows % 2 == 0]
        odd_rows = rows[rows % 2 == 1]
        
        new_lattice[even_rows] = previous_lattice
        new_lattice[odd_rows] = generate(odd_rows)
    
    return new_lattice

def shift_points(s: int, points: np.array):
    shift_vector = np.random.uniform(0, 1, s)
    
    return np.modf(points + shift_vector)[0]

In [152]:
def generate_sobol_points(
    s: int,
    m: int,
    scramble=True,
):
    sampler = qmc.Sobol(d=s, scramble=scramble)
    return sampler.random_base2(m=m)

In [355]:
def riemann_product_rule(
    N,  # number of intervals
    s,  # dimension
):
    h = 1 / N  # interval size
    n = N + 1  # number of points
    
    points = np.meshgrid(*[np.linspace(0, 1, n) for _ in range(s)])
    points = np.array(points).T.reshape(-1, s)
    
    weights = np.full(points.shape[0], h ** s)
    
    return np.sum(f(s, points) * weights)

In [352]:
s = 10
z = generating_vectors[s]
m = 10

In [353]:
points = generate_lattice_points(z, m)
np.mean(f(s, points))

0.9996513237528162

In [354]:
points = generate_sobol_points(s, m, scramble=False)
np.mean(f(s, points))

1.0396389683923086

In [363]:
riemann_product_rule(50, 4)

1.0879442561108281

### Part (a)

Compute the error for different $N$ and $s$ and compare the results for the product rules from Exercise Sheet 1.

### Part (b)

In [181]:
R = [8, 16, 32]

In [164]:
points = generate_lattice_points(z, m)
estimates = []

for _ in range(N_sample):
    shifted_points = shift_points(s, points)
    estimates.append(
        np.mean(f(s, shifted_points))
    )
    
lattice_estimate = np.mean(np.array(estimates))

In [165]:
latice_error = np.abs(true_value - lattice_estimate)
latice_error

0.001553888962271932

In [167]:
estimates = []

for _ in range(N_sample):
    scrambled_points = generate_sobol_points(s, m, scramble=True)
    estimates.append(
        np.mean(f(s, scrambled_points))
    )
    
sobol_estimate = np.mean(np.array(estimates))

In [169]:
sobol_error = np.abs(true_value - sobol_estimate)
sobol_error

0.0010301615550634136